In [ ]:
# ============================================================
# UMAP Harmony — FINAL (con Level2_final: Conv_T_other -> CD4_Memory)
# - NO lee X ni layers (evita MemoryError de /layers/log1p_10k)
# - Lee SOLO: obs + obsm['X_pca'] desde el .h5ad vía h5py
# - Ejecuta Harmony con key='patientID'
# - Calcula neighbors + UMAP sobre X_pca_harmony
# - Guarda UMAPs: Level1_refined, Level2_final, disease, patientID en figures_final/
# - QA numérico usa Level2_final (no Level2 legacy)
# ============================================================

# ---------------------------
# CELL 1 — imports + paths + lectura mínima (obs + X_pca) + Level2_final
# ---------------------------
from pathlib import Path
import json
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
import h5py

def _read_elem(h5obj):
    """
    Lee un elemento AnnData desde h5py sin materializar /layers.
    Funciona en anndata>=0.8 (rutas pueden variar).
    """
    try:
        from anndata.experimental import read_elem as _re
        return _re(h5obj)
    except Exception:
        try:
            from anndata._io.specs import read_elem as _re
            return _re(h5obj)
        except Exception as e:
            raise ImportError(
                "No puedo importar read_elem (anndata.experimental.read_elem / anndata._io.specs.read_elem). "
                "Necesitas anndata con soporte de lectura de elementos."
            ) from e


NOTEBOOK_DIR = Path.cwd()

def find_project_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / "data_processed").exists():
            return p
    raise FileNotFoundError(f"No encuentro 'data_processed' subiendo desde: {start}")

PROJECT_ROOT = find_project_root(NOTEBOOK_DIR)
DATA_PROCESSED = PROJECT_ROOT / "data_processed"

IN_PATH = DATA_PROCESSED / "TFM_CIRRHOSIS_main_filtered_for_analysis.h5ad"

OUT_FIG = PROJECT_ROOT / "figures_final"
OUT_SUM = PROJECT_ROOT / "summary_tables_final"
OUT_FIG.mkdir(exist_ok=True)
OUT_SUM.mkdir(exist_ok=True)

print("Scanpy:", getattr(sc, "__version__", "unknown"))
print("PROJECT_ROOT:", PROJECT_ROOT)
print("IN_PATH     :", IN_PATH)
print("OUT_FIG     :", OUT_FIG)
print("OUT_SUM     :", OUT_SUM)

if not IN_PATH.exists():
    raise FileNotFoundError(f"No existe IN_PATH:\n{IN_PATH}")

# --- cargar mapa Level2_final (ya lo generaste) ---
MAP_PATH = OUT_SUM / "Level2_final_map.json"
print("MAP_PATH    :", MAP_PATH)
if not MAP_PATH.exists():
    raise FileNotFoundError(
        f"No existe MAP_PATH:\n{MAP_PATH}\n"
        "Este notebook requiere el mapa Level2_final_map.json (Conv_T_other -> CD4_Memory)."
    )

with open(MAP_PATH, "r", encoding="utf-8") as f:
    level2_map = json.load(f)

needed_obs = ["patientID", "disease", "Level1_refined", "Level2"]

with h5py.File(IN_PATH, "r") as f:
    # obs (DataFrame)
    if "obs" not in f:
        raise KeyError("No existe grupo /obs en el .h5ad (archivo corrupto o no estándar).")

    obs_full = _read_elem(f["obs"])
    if not isinstance(obs_full, pd.DataFrame):
        raise TypeError(f"read_elem(/obs) no devolvió DataFrame. Tipo: {type(obs_full)}")

    missing = [c for c in needed_obs if c not in obs_full.columns]
    if missing:
        raise KeyError(f"Faltan columnas en obs: {missing}")

    obs_full = obs_full.copy()
    obs_full["patientID"] = obs_full["patientID"].astype(str)
    obs_full["disease"] = obs_full["disease"].astype(str)
    obs_full["Level1_refined"] = obs_full["Level1_refined"].astype(str)

    # Mantener Level2 como object (sin convertir NaN->"nan")
    l2_obj = obs_full["Level2"].astype("object")

    # Level2_final (mapping Conv_T_other -> CD4_Memory)
    l2_final = l2_obj.replace(level2_map)
    obs_full["Level2_final"] = pd.Categorical(l2_final)

    print("\n[CHECK] Conv_T_other remaining en Level2_final (debe ser 0):",
          int((obs_full["Level2_final"].astype(str) == "Conv_T_other").sum()))
    print("[CHECK] CD4_Memory count en Level2_final:",
          int((obs_full["Level2_final"].astype(str) == "CD4_Memory").sum()))

    # RBC-out sanity (si aparece, se excluye SOLO para UMAP)
    keep = ~(
        (obs_full["Level1_refined"].astype(str) == "RBC") |
        (l2_obj.astype(str) == "RBC") |
        (obs_full["Level2_final"].astype(str) == "RBC")
    )
    n_rbc = int((~keep).sum())
    if n_rbc > 0:
        print(f"[WARN] Aún hay RBC en el objeto ({n_rbc} células). Para UMAP Harmony se excluirán.")

    idx_keep = np.flatnonzero(keep.to_numpy(dtype=bool))
    obs_keep = obs_full.iloc[idx_keep].copy()

    # X_pca
    if "obsm" not in f or "X_pca" not in f["obsm"]:
        raise KeyError("No existe /obsm/X_pca en el .h5ad. Necesitas PCA calculado antes.")

    X_pca_ds = f["obsm"]["X_pca"]
    X_pca = np.asarray(X_pca_ds[idx_keep, :], dtype=np.float32)

print("\nX_pca shape:", X_pca.shape)
print("n_obs total :", obs_full.shape[0])
print("n_obs keep  :", obs_keep.shape[0])

# Construir AnnData mínimo (NO trae X/layers)
adata_umap = ad.AnnData(
    X=np.zeros((obs_keep.shape[0], 1), dtype=np.float32),
    obs=obs_keep
)

# --- fijar nº PCs a usar y recortar X_pca para que Harmony sea consistente ---
N_PCS = min(50, X_pca.shape[1])
adata_umap.obsm["X_pca"] = X_pca[:, :N_PCS].astype(np.float32, copy=False)

print("\nAnnData mínimo para Harmony:", adata_umap)
print("obs columns:", adata_umap.obs.columns.tolist())
print("X_pca used shape:", adata_umap.obsm["X_pca"].shape)


# ---------------------------
# CELL 2 — Harmony sobre X_pca (batch=patientID) + neighbors + UMAP
# ---------------------------
try:
    import scanpy.external as sce
    HAVE_SCE = True
except Exception:
    HAVE_SCE = False

try:
    import harmonypy as hm
    HAVE_HM = True
except Exception:
    HAVE_HM = False

if not HAVE_SCE and not HAVE_HM:
    raise ModuleNotFoundError(
        "No encuentro ni scanpy.external (harmony_integrate) ni harmonypy.\n"
        "Instala harmonypy:\n"
        "  conda install -c conda-forge harmonypy\n"
        "o:\n"
        "  python -m pip install -U harmonypy"
    )

BATCH_KEY = "patientID"
N_NEIGHBORS = 15
RANDOM_STATE = 0

print("\nParams:")
print("BATCH_KEY   :", BATCH_KEY)
print("N_PCS       :", N_PCS)
print("N_NEIGHBORS :", N_NEIGHBORS)

adata_umap.obs[BATCH_KEY] = adata_umap.obs[BATCH_KEY].astype("category")

if HAVE_SCE and hasattr(sce.pp, "harmony_integrate"):
    print("\n[Harmony] usando scanpy.external.pp.harmony_integrate ...")
    sce.pp.harmony_integrate(
        adata_umap,
        key=BATCH_KEY,
        basis="X_pca",
        adjusted_basis="X_pca_harmony",
        max_iter_harmony=20,
    )
else:
    print("\n[Harmony] usando harmonypy.run_harmony (fallback) ...")
    Z = adata_umap.obsm["X_pca"].T  # (pcs x cells)
    meta = adata_umap.obs[[BATCH_KEY]].copy()
    ho = hm.run_harmony(Z, meta, vars_use=[BATCH_KEY])
    adata_umap.obsm["X_pca_harmony"] = ho.Z_corr.T.astype(np.float32, copy=False)

print("X_pca_harmony shape:", adata_umap.obsm["X_pca_harmony"].shape)

sc.pp.neighbors(
    adata_umap,
    n_neighbors=N_NEIGHBORS,
    n_pcs=N_PCS,
    use_rep="X_pca_harmony",
)
sc.tl.umap(adata_umap, random_state=RANDOM_STATE)

adata_umap.obsm["X_umap_harmony"] = adata_umap.obsm.pop("X_umap")
print("[OK] Harmony + neighbors + UMAP listo.")
print("obsm keys:", list(adata_umap.obsm.keys()))

# Guardar coords para downstream (QA coherence / compactness)
emb = pd.DataFrame(
    adata_umap.obsm["X_umap_harmony"],
    index=adata_umap.obs_names,
    columns=["UMAP1_harmony", "UMAP2_harmony"]
)

emb_out = pd.concat(
    [
        adata_umap.obs[["patientID", "disease", "Level1_refined", "Level2", "Level2_final"]],
        emb
    ],
    axis=1
)

emb_path = OUT_SUM / "UMAP_Harmony_embeddings.csv"
emb_out.to_csv(emb_path, index=True)
print("Saved embeddings:", emb_path)


# ---------------------------
# CELL 3 — plots (Level1_refined, Level2_final, disease, patientID)
# ---------------------------
sc.settings.autoshow = False

def save_umap(color, out_name, title=None, legend_loc="right margin", legend_fontsize=7):
    ax = sc.pl.embedding(
        adata_umap,
        basis="umap_harmony",
        color=color,
        show=False,
        frameon=False,
        legend_loc=legend_loc,
        legend_fontsize=legend_fontsize,
        title=title,
    )
    fig = ax.figure
    out_path = OUT_FIG / out_name
    fig.savefig(out_path, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("Saved:", out_path)

save_umap(
    color="Level1_refined",
    out_name="UMAP_Harmony_Level1_refined.png",
    title="UMAP (Harmony; batch=patientID) — Level1_refined",
    legend_loc="right margin",
    legend_fontsize=7,
)

save_umap(
    color="Level2_final",
    out_name="UMAP_Harmony_Level2.png",
    title="UMAP (Harmony; batch=patientID) — Level2_final",
    legend_loc="right margin",
    legend_fontsize=6,
)

save_umap(
    color="disease",
    out_name="UMAP_Harmony_disease.png",
    title="UMAP (Harmony; batch=patientID) — disease",
    legend_loc="right margin",
    legend_fontsize=8,
)

# patientID: sin leyenda (si no, la figura explota)
save_umap(
    color="patientID",
    out_name="UMAP_Harmony_patientID.png",
    title="UMAP (Harmony; batch=patientID) — patientID",
    legend_loc="none",
    legend_fontsize=6,
)

print("\n[OK] UMAPs Harmony guardados en figures_final/.")


# ============================================================
# CELL X — QA numérico de UMAP/vecinos (usa Level2_final)
# ============================================================
import scipy.sparse as sp

QA_DIR = OUT_SUM
QA_DIR.mkdir(exist_ok=True)

def _topk_neighbors_from_connectivities(C, k=15):
    if not sp.issparse(C):
        C = sp.csr_matrix(C)
    C = C.tocsr()
    n = C.shape[0]
    neigh_idx = []
    for i in range(n):
        start, end = C.indptr[i], C.indptr[i+1]
        cols = C.indices[start:end]
        data = C.data[start:end]
        m = cols != i
        cols = cols[m]
        data = data[m]
        if cols.size == 0:
            neigh_idx.append(np.array([], dtype=int))
            continue
        if cols.size > k:
            sel = np.argpartition(-data, k-1)[:k]
            cols = cols[sel]
            data = data[sel]
            ord2 = np.argsort(-data)
            cols = cols[ord2]
        neigh_idx.append(cols.astype(int))
    return neigh_idx

def neighbor_mixing_stats(labels: np.ndarray, neigh_idx, k_expected=15):
    labels = labels.astype(str)
    n = len(labels)
    fracs = np.full(n, np.nan, dtype=float)
    for i in range(n):
        nb = neigh_idx[i]
        if nb.size == 0:
            continue
        fracs[i] = np.mean(labels[nb] == labels[i])
    return {
        "n_cells": int(n),
        "k_expected": int(k_expected),
        "mean_same": float(np.nanmean(fracs)),
        "median_same": float(np.nanmedian(fracs)),
        "p25_same": float(np.nanpercentile(fracs, 25)),
        "p75_same": float(np.nanpercentile(fracs, 75)),
        "na_cells": int(np.isnan(fracs).sum()),
    }

def neighbor_entropy(labels: np.ndarray, neigh_idx):
    labels = labels.astype(str)
    uniq = np.unique(labels)
    L = len(uniq)
    if L <= 1:
        return {"mean_entropy_norm": 0.0, "L": int(L)}
    ent = []
    for nb in neigh_idx:
        if nb.size == 0:
            continue
        vals = labels[nb]
        _, cts = np.unique(vals, return_counts=True)
        p = cts / cts.sum()
        h = -np.sum(p * np.log(p + 1e-12))
        ent.append(h / np.log(L))
    return {"mean_entropy_norm": float(np.mean(ent)) if len(ent) else np.nan, "L": int(L)}

if "connectivities" not in adata_umap.obsp:
    raise KeyError("No encuentro adata_umap.obsp['connectivities']. ¿Has corrido sc.pp.neighbors(...) antes?")

C = adata_umap.obsp["connectivities"]
k_expected = int(adata_umap.uns["neighbors"]["params"].get("n_neighbors", 15))
neigh_idx = _topk_neighbors_from_connectivities(C, k=k_expected)

qa_rows = []
for col in ["patientID", "disease", "Level1_refined", "Level2_final"]:
    lab = adata_umap.obs[col].astype(str).to_numpy()
    st = neighbor_mixing_stats(lab, neigh_idx, k_expected=k_expected)
    ent = neighbor_entropy(lab, neigh_idx)
    qa_rows.append({"label": col, **st, **ent})

qa = pd.DataFrame(qa_rows)
qa_path = QA_DIR / "QA_UMAP_Harmony_neighbor_mixing.csv"
qa.to_csv(qa_path, index=False)

print("\nSaved:", qa_path)
print(qa.to_string(index=False))

# Matriz paciente->paciente
patients = adata_umap.obs["patientID"].astype(str).to_numpy()
uniq_p = np.unique(patients)
p_to_i = {p: i for i, p in enumerate(uniq_p)}
M = np.zeros((len(uniq_p), len(uniq_p)), dtype=np.float64)

for i in range(adata_umap.n_obs):
    p_i = p_to_i[patients[i]]
    nb = neigh_idx[i]
    if nb.size == 0:
        continue
    for j in nb:
        p_j = p_to_i[patients[j]]
        M[p_i, p_j] += 1.0

row_sums = M.sum(axis=1, keepdims=True)
M_norm = np.divide(M, np.maximum(row_sums, 1.0))

mix_mat = pd.DataFrame(M_norm, index=uniq_p, columns=uniq_p)
mix_path = QA_DIR / "QA_UMAP_Harmony_patient_neighbor_mixing_matrix.csv"
mix_mat.to_csv(mix_path)
print("\nSaved:", mix_path)

# Dominancia por Level2_final
tmp = adata_umap.obs[["patientID", "disease", "Level2_final", "Level1_refined"]].copy()
tmp["patientID"] = tmp["patientID"].astype(str)
tmp["Level2_final"] = tmp["Level2_final"].astype(str)

ct = (tmp.groupby(["Level2_final", "patientID"]).size()
      .reset_index(name="n_cells"))

def _gini(x):
    x = np.asarray(x, dtype=float)
    if x.size == 0:
        return np.nan
    if np.all(x == 0):
        return 0.0
    x = np.sort(x)
    n = x.size
    cumx = np.cumsum(x)
    g = (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n
    return float(g)

lvl2_stats = []
for lvl2, sub in ct.groupby("Level2_final"):
    counts = sub["n_cells"].to_numpy()
    lvl2_stats.append({
        "Level2_final": lvl2,
        "n_patients_with_cells": int((counts > 0).sum()),
        "total_cells": int(counts.sum()),
        "max_patient_share": float(counts.max() / counts.sum()) if counts.sum() > 0 else np.nan,
        "gini_patient_counts": _gini(counts),
    })

lvl2_stats = pd.DataFrame(lvl2_stats).sort_values(
    ["max_patient_share", "gini_patient_counts"], ascending=False
)

lvl2_path = QA_DIR / "QA_Level2_final_patient_dominance.csv"
lvl2_stats.to_csv(lvl2_path, index=False)

lvl2_path_legacy = QA_DIR / "QA_Level2_patient_dominance.csv"
lvl2_stats.rename(columns={"Level2_final": "Level2"}).to_csv(lvl2_path_legacy, index=False)

print("\nSaved:", lvl2_path)
print("Saved (legacy alias):", lvl2_path_legacy)
print("\n[OK] QA numérico Harmony completado.")


# ---------------------------
# CELL Z — verificación de PNGs + RBC-out real (SIN sc.read_h5ad)
# ---------------------------
FIG_DIR = OUT_FIG
expected = [
    "UMAP_Harmony_Level1_refined.png",
    "UMAP_Harmony_Level2.png",
    "UMAP_Harmony_disease.png",
    "UMAP_Harmony_patientID.png",
]

missing = []
for fn in expected:
    p = FIG_DIR / fn
    ok = p.exists()
    size_mb = (p.stat().st_size / 1e6) if ok else 0
    print(f"{fn:40s} exists={ok} sizeMB={size_mb:.2f}")
    if not ok:
        missing.append(fn)

if missing:
    raise FileNotFoundError("FALTAN PNGs UMAP Harmony: " + ", ".join(missing))

# RBC-out check leyendo SOLO obs vía h5py (sin tocar /layers)
with h5py.File(IN_PATH, "r") as f:
    obs_z = _read_elem(f["obs"])
    l1r = set(obs_z["Level1_refined"].astype(str).unique()) if "Level1_refined" in obs_z.columns else set()
    l2  = set(obs_z["Level2"].astype(str).unique()) if "Level2" in obs_z.columns else set()

if "RBC" in l1r or "RBC" in l2:
    raise RuntimeError(
        f"RBC sigue presente en OUT_FILTER "
        f"(Level1_refined has RBC? {'RBC' in l1r} | Level2 has RBC? {'RBC' in l2})"
    )

print("[OK] UMAP Harmony: PNGs presentes + RBC-out verificado (sin cargar layers).")

“Algunas subpoblaciones (p.ej. ISG_Myeloid, aDC…) muestran fuerte dominancia por paciente (max_patient_share alto), por lo que su interpretación debe hacerse con cautela y priorizando análisis a nivel de paciente (composición scCODA/pseudobulk).”